# Define constants

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
# %matplotlib inline

input_path = './chest_Xray/'

img_dims = 150
epochs = 20
batch_size = 96

def process_data(img_dims, batch_size):
    train_datagen = ImageDataGenerator(
        rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    # This is fed to the network in the specified batch sizes and image dimensions
    # Allows for faster and lighter imports
    train_gen = train_datagen.flow_from_directory(
        directory=input_path+'train',
        target_size=(img_dims, img_dims),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True)

    # Using test data for validation as it's bigger
    test_gen = test_datagen.flow_from_directory(
        directory=input_path+'test',
        target_size=(img_dims, img_dims),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True)

    test_data = []
    test_labels = []

    for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = plt.imread(input_path + 'test' + cond + img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond == '/NORMAL/':
                label = 0
            elif cond == '/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)

    test_data = np.array(test_data)
    test_labels = np.array(test_labels)

    return train_gen, test_gen, test_data, test_labels

# Import and process data

In [2]:
for _set in ['train', 'val', 'test']:
    n_normal = len(os.listdir(input_path + _set + '/NORMAL'))
    n_infect = len(os.listdir(input_path + _set + '/PNEUMONIA'))
    print('Set: {}, normal images: {}, pneumonia images: {}'.format(
        _set, n_normal, n_infect))

train_gen, test_gen, test_data, test_labels = process_data(
    img_dims, batch_size)

Set: train, normal images: 1341, pneumonia images: 3875
Set: val, normal images: 8, pneumonia images: 8
Set: test, normal images: 234, pneumonia images: 390
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


# Setup CNN

### Using 5 convolution Layers and 3 fully connected Layers

In [4]:

from tensorflow.keras.models import Model
from tensorflow.keras import layers

# Input Layer
inputs = layers.Input(shape=(img_dims, img_dims, 3))

# First Conv Layer
x = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(inputs)
x = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPool2D()(x)

# Second Conv Layer
x = layers.SeparableConv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.SeparableConv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)

# Third Conv Layer
x = layers.SeparableConv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.SeparableConv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)

# Fourth Conv Layer
x = layers.SeparableConv2D(128, (3, 3), activation="relu", padding="same")(x)
x = layers.SeparableConv2D(128, (3, 3), activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
x = layers.Dropout(0.2)(x)

# Fifth Conv Layer
x = layers.SeparableConv2D(256, (3, 3), activation="relu", padding="same")(x)
x = layers.SeparableConv2D(256, (3, 3), activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
x = layers.Dropout(0.2)(x)

# Fully Connected Layer
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.7)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)

# Output Layer
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# Define Callbacks

In [5]:
from Class.callback import EarlyStoppingAtMinLoss
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

checkpoint = ModelCheckpoint(
    filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(
    monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(
    monitor='val_loss', min_delta=0.1, patience=4, mode='min')
# early_stop = EarlyStoppingAtMinLoss(4)

# Train CNN

### Use train and validate datasets

In [6]:
hist = model.fit(
    train_gen, steps_per_epoch=train_gen.samples // batch_size,
    epochs=epochs, validation_data=test_gen,
    validation_steps=test_gen.samples // batch_size, callbacks=[checkpoint, lr_reduce])

Epoch 1/20
 1/54 [..............................] - ETA: 3:11 - loss: 0.6933 - accuracy: 0.4688

KeyboardInterrupt: 

# Testing Metrics

### Accuracy and Loss graphs

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

### Model metrics - Accuracy, F1, Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix

preds = model.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('TEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
f = 2 * precision * recall/(precision + recall)
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(f))

print('\nTRAIN METRIC ----------------------')
print('Train acc: {}'.format(np.round((hist.history['accuracy'][-1])*100, 2)))

print('\nCONFUSION MATRIX ------------------')
print(cm)
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_percentages = ["{0:.2%}".format(value) for value in cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v3}" for v1, v3 in zip(group_names,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

# Save model

In [ ]:
model.save('./saved_models_v2/' + str(epochs) + '_adam_b' + str(batch_size) + '_a' + str(int(acc)) + '_f' + str(int(f)))